In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
from pathlib import Path

# Ensure the notebook can find the 'src' directory
sys.path.append(os.path.abspath(os.path.join('..')))

In [3]:
import mlflow

# Use resolve() to get the actual, absolute path on your Mac
# This points to: /Users/davideferreri/.../renewable-energy-forecast/mlflow_data
absolute_project_root = Path.cwd().parent.resolve()
absolute_mlflow_dir = absolute_project_root / "mlflow_data"

# Create the directory if it doesn't exist
absolute_mlflow_dir.mkdir(parents=True, exist_ok=True)

# Set the URI using the absolute path
mlflow.set_tracking_uri(f"file://{absolute_mlflow_dir}")
mlflow.set_experiment("Solar_Power_Forecasting")

print(f"Tracking to: {mlflow.get_tracking_uri()}")

/Users/davideferreri/Documents/GitHub/renewable-energy-forecast/.venv/lib/python3.11/site-packages/mlflow/tracking/_tracking_service/utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/01/12 16:09:26 INFO mlflow.tracking.fluent: Experiment with name 'Solar_Power_Forecasting' does not exist. Creating a new experiment.


Tracking to: file:///Users/davideferreri/Documents/GitHub/renewable-energy-forecast/mlflow_data


In [4]:
# 1. Imports
from src.preprocessing.data_preprocessing import load_and_merge_data
from src.features.build_features import create_features
from src.models.train import train_model

# 2. Ingest
# This defines 'df' so the next line can use it
df = load_and_merge_data("Plant_1_Generation_Data.csv", "Plant_1_Weather_Sensor_Data.csv")

# 3. Feature engineernig
df_featured = create_features(df)

# 4. Inspect the model output
model, model_mae = train_model(df_featured)
print("-" * 30)
print(f"BASELINE MAE: 237.23 kW")
print(f"MODEL MAE:    {model_mae:.2f} kW")
print("-" * 30)

if model_mae < 237.23:
    print("🚀 SUCCESS: The model beat the persistence baseline!")
else:
    print("⚠️ WARNING: The model is worse than the baseline. Needs more feature engineering.")

/Users/davideferreri/Documents/GitHub/renewable-energy-forecast/src/preprocessing/data_preprocessing.py:25: UserWarning: Parsing dates in %d-%m-%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  gen_df['DATE_TIME'] = pd.to_datetime(gen_df['DATE_TIME'], dayfirst=False)
2026/01/12 16:09:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


✅ Training Complete. MAE: 166.80 kW
------------------------------
BASELINE MAE: 237.23 kW
MODEL MAE:    166.80 kW
------------------------------
🚀 SUCCESS: The model beat the persistence baseline!


In [5]:
import joblib

# Define the models directory (at project root)
model_dir = Path.cwd().parent / "models"
model_dir.mkdir(exist_ok=True)

# Save the model
model_path = model_dir / "solar_model.joblib"
joblib.dump(model, model_path)

print(f"✅ Model saved to: {model_path}")

✅ Model saved to: /Users/davideferreri/Documents/GitHub/renewable-energy-forecast/models/solar_model.joblib
